# Lab 7: PID Control

In previous lab sheets we have developed the following understanding and functionality:
- Familiarised with the Arduino IDE, Sketch files, uploading to the 3Pi+, and utilising the Serial interface
- Implemented basic operation of the 3Pi+ motors, and encapsulated this within safe and confident function(s), and utilised a `class`.
- Implemented basic read functionality of the 3 central IR Line sensors facing the work surface, and encapsulated this within function(s).
- Explored the use of `millis()` to approximate task-scheduling on the 3Pi+.
- Developed a bang-bang controller, using logic to control the robot.
- Implement a calibration routine for the ground sensor.
- Developed a weighted-measurement of the line sensors, calculated an `error signal`, and used a proportional controller to steer the robot to produce line-following behaviour. 
- Completed `interrupt service routine` code to count the number of encoder pulses as the wheels rotate.
- Developed kinematics and odometry to track the robot position over time.  


In this labsheet we will investigate a PID controller.  A PID controller can be used in multiple contexts in the line following challenge:
- to control the speed of the wheel rotation, so that they become a more consistent and reliable subsystem.
- to control the heading of the robot, with respect to line following.
- to control the heading of the robot, with respect to $\theta$ in kinematics, allowing for good straight-line travel.
- to control the rotation of the robot, with respect to $\theta$ in kinematics, in order to make precise turn motions.
- to control the travel velocity of the robot, with respect to the distance from origin in kinematics, to allow the robot to return to home in a controlled manner. 

Whilst this may seem like a long list, the PID approach is quite general and you should find it relatively easy to re-apply the techinque into different contexts.  This is one of the strengths of PID control.  

To work on PID, we will first implement a measure of rotation velocity of the 3Pi+ wheels.  With a measure of rotation velocity, we will then tune a PID controller step-by-step.


<hr><br><br><br><br>

# Rotation Velocity Estimation

In previous labsheets you have developed code to:
- for various tasks, calculate a period of elapsed time, in milliseconds and microseconds, using `millis()` and `micros()`.
- developed code to effect encoder counts, and to use encoder counts for kinematics.

In this exercise both of these componenets will be used to estimate the rotation velocity of a wheel.  Recall that:
<p align="center">
$v = d / \Delta t$
</p>

Where $v$ is velocity, $d$ is distance, and $\Delta t$ is the change in time.  

On the 3Pi+ we actually have three reasonable locations to implement an estimation of rotation velocity:
1. Within `loop()`, allowing for some variation of time elapsed $\Delta t$ via `millis()` or `micros()`, and recording the difference in encoder counts as $d$.
2. Within the encoder ISRs.  Here, $d$ would always equal 1, because the ISR activates with every rotation of the wheel.  $\Delta t$ would need to be captured.
3. Within a `Timer` ISR, where the `Timer` would be configured so that $\Delta t$ could be fixed to a known period, and the change in encoder counts would be $d$.

In the following exercise, we will adopt the `loop()` method because it is quick to implement and sufficient to complete the line following challenge.  The other two are left to your curiousity.

## Exercise 1: Rotation Velocity Estimation

**Decompose the problem:** In this exercise, follow the normal development of working with just 1 wheel to begin with, and then update your code for both wheels when it is appropriate.

1. **Hypothesise:** What is a serious limitation of implementing the velocity estimate within the encoder ISR?  This is a tricky question!

2. It is recommended to start a new sketch to keep your program as simple as possible.  Include your code to operate the `encoders`.  Once you get this working and refactor your code, you might want to merge it back in with your main software.
  - Implement some `global variables` to track the change of encoder counts for each iteration of `loop()`.
  - Implement some `global variables` to track the change of time per iteration of `loop()`.
  - Implement a `global variable` to store an estimation of rotation velocity.

3. Implement code within `loop()` to:
  - Capture a change of encoder counts.  Remember to pay attention to your variable `data types`.
  - It is not strictly necessary to convert the change in encoder counts to radians or mm, this is at your discretion.
  - Capture the time elapsed between `loop()` iterations.
  - Divide the change of encoder by the time elapsed.
  - Store the result into your global rotation velocity variable.  Remember to think about a suitable `data type`.
  - **Validate:** Use Serial to view your rotation velocity estimation.  You may see a very rough looking signal when you move the wheel by hand.  
  - **Validate:** Use your motor command functions to see how the signal appears at different motor power settings.

4. Either use `delay()` or `millis()` (within an `if()` statement) to investigate how the frequency of calculation this velocity estimate changes quality of the signal.
  - **Hypothesise:** You may see a very spikey or scattered plot. What might be an underlying cause of this?

5. **Refactor:** Before you move on to the second wheel, consider updating your code to either be contained within a `function` or within a `class`.

6. Consider implementing a `low pass filter` for your velocity estimate signal.  This can be quickly and crudely achieved with the following code extract:
  - **Hypothesise:** What is an immediately obvious disadvantage to a low pass filter as provided below?

```c
// An initialised valued
float lpf = 0;
float alpha = 0.2; // value between 0:1
                   // how much weight to new readings?

// Update low pass filter
lpf = ( lpf * (1 - alpha ) ) + ( new_reading * alpha );

```


# PID Control Overview


In terms of our robot performance, PID is interesting because it may offer smoother control over the robot motion.  This can be achieved in two ways:
1. Effective control of the wheels speeds.
2. Effective control of the heading of the robot.

In both cases, it is desirable that our robot does not have `jerk` in the motion - this is when the robot makes sudden and/or large changes in the motion.  For our small robot, rapid changes to wheel speed are likely to cause the wheels to slip on the surface.  From the last labsheet, we have been using the rotation of the wheels to estimate the robots `pose` (position and rotation).  Therefore, if the wheels slip on the surface, there will be encoder counts but the robot will not change it's `pose` in reality.  Within the line following challenge, wheel slip will severely effect the ability for your robot to complete the final objective, `return to home`.  

In the above two use cases (wheel speed and heading), we can imagine that rapid changes to heading are likely to cause a dramatic change of required wheels speeds.  This is because the wheel speed will be responsible for orientating the robot to a new heading.  Therefore, it is desirable for our heading to change in a continuous manner (not sudden), just as it is desirable for wheel speed to change in a continuous manner.  

PID will allow us to utilise a continuous control signal relative to some measurement of the system.  The measurement could come from:
- kinematics, such as theta (rotation) or a point to travel to (x,y)
- the position of the line under the robot
- the speed of rotation of the wheels

A nice example of PID being used for smooth motion control is the following video of an e-puck robot tracking a ball with a low-resolution camera: <a href="https://www.youtube.com/watch?v=Pga71leqf1A">YouTube</a>.  Whilst not documented, it appears at least two PID controllers are being used:
(1) tracking left to right position of ball, (2) tracking a set distance from the ball. 




<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/PID_Overview.png?raw=true">
</p>

The above diagram illustrates the general form of a PID controller.  Outside of the blue-dotted box, we can see that a measurement from the system output, $y(t)$ is combined with a desired measurement, $r(t)$, to form the error signal, $e(t)$:

> $e(t) = r(t) - y(t)$


In a previous labsheet, you will have already implemented a `proportional controller` (P-controller) for line following, one of the three P, I and D elements inside the blue-dotted box.  The `integral` and `derivative` components were not investigated.  

The P-Controller implementation allows us to address many parts of the above diagram:
- the **`measurement`**, $y(t)$, was the output from the weighted-measurement calculation, which estimated the line position under the ground sensor in the range [ -1.0 : +1.0 ].
- the **`demand`**, $r(t)$, was set to 0, because it was desired to have the line positioned central between the ground sensors, which equated to a measurement value of 0.  Sometimes `demand` is also known as `setpoint`.
- the **`error signal`**, $e(t)$, meaning the difference between the measurement and the demand, was therefore $e(t) = 0 - y(t)$.
- the **`feedback signal`**, $u(t)$, was calculated by multiplying the error signal by a maximum motor power.  This had the effect of proportionally scaling motor power with respect to the error.
- therefore, the **`proportional gain`**, $K_{p}$, was set as the maximum motor power.  
- the `process` box above was the motor and wheel subsystem of the 3Pi+ robot.  In receiving the feedback signal (`analogWrite()`), the system process was altered, and so the measurement of the line sensors was effected.

We can make some interesting observations:
- the `demand` and the `measurement` were of the same units, because the PID controller is aiming to minimise the `error signal` to 0.  
- the `feedback signal`, $u(t)$, is of different units.  In the line following implementation, the feedback signal was an 8-bit value [0:255] sent to the Arduino function `analogWrite()` to send power to the motor.
- the PID controller is therefore trying to match $r(t)$ and $y(t)$ by manipulating the feedback signal, $u(t)$.
- the effect of the feedback signal $u(t)$ is to effect physical interaction of the robot with the environment, resulting in the new measurement $y(t)$.

In general terms, we can consider the PID controller as acting to transform the error signal into a feedback signal.  The question for a PID controller designer is:
> How can the error signal be usefully transformed into a feedback signal?

In the above diagram we also see the identities $K_{p}$, $K_{i}$ and $K_{d}$.  These are the system `gains`, numerical constants used to manipulate the error signal.  These are the internal parts of the PID algorithm that require a human designer to `tune` (calibrate), to create the useful transformation of error signal to feedback signal.  

The `proportional` component gain, $K_{p}$, is the easiest to gain to consider.  $K_{p}$ simply multiplies with error signal $e(t)$.  If this were a line on a graph, $K_{p}$ would effect the slope of the line.  Therefore, $K_{p}$ will effectively scale a given value of $e(t)$ to produce the feedback signal $u(t)$.




## Exercise 2: Implement a P-Controller

1. Review and update the `pid.h` source file in the template provided on Github (<a href="https://github.com/paulodowd/EMATM0053_21_22/tree/main/3PI_CodeStub">Github Page</a>, <a href="https://github.com/paulodowd/EMATM0053_21_22/raw/main/3PI_CodeStub/Labsheet_X.zip">Download Link for Zip file</a>):
  - Implement variables within the class to store the `p_term`, and the `feedback signal`.
  - Utilise an `initialise()` function to set the class variables to 0.
  - Utilise the `update()` function to implement the P-Controller update:
    - Implement the `error signal` calculation, $e(t) = r(t) - y(t)$.
    - Implement the `p-term`, of the form $ p = K_{p} * e $
    - Return the `p-term` as the feedback signal.
  -


<hr><br><br><br><br>

<hr>
# Recommended Reading

Quantisation Error: <a href="">DSP Guide for Engineers Chapter 3.</a>

# Concerning PID Controllers:

A PID controller can be used to control all sorts of systems.  For instance, the temperature of a room: 
- the **measurement** would be the sensed room temperature.
- the **output** would be a heating element, to effect the temperature of the room.
- the **demand** input would be the user temperature control on the wall.

Therefore the operation of a PID controller can be described in natural language as:
- **measuring** the temperature of the room
- attempting to match the measurement to the user **demand** (e.g., what is the difference, the **error**)
- adjusting the **output** heating element _appropriately_ to be hotter or cooler, to raise or lower the temperature of the room.

It is important to note that the measurement, output and demand can each be of different units and of different scales:
- The user demand might be in Celcius in 1 degree increments in the range [5 : 30].
- The measurement might be a 12-bit digital value from a linear temperature sensor
- The output might be a high electical-current to drive a non-linear industrial heater working in kilo-Watts.  

Therefore, a PID controller must map a **relationship** between demand, output and measurement (e.g., room gets too hot -> heater must get cooler).  Furthermore, there is a necessary **transformation** that must occur between:
- making a comparison between measurement and demand to determine the **error signal**
- transforming the error signal to correct the **output**.

In the above example of maintaining the temperature of a room, we would need to transform a **small error signal** (between a temperature sensor and user demand), to a **large voltage** for a kilo-Watt heater.  For controlling the motor on the Romi, we start with an **encoder count** (`unsigned long`), and we must transform this into an **analogWrite()** (`[0:255]`).  Every system will:
- have a different processing of demand, measurement, error signal and output.
- the relationship between demand and output will be different. 
- the transformation from error signal to output will be different.
- the actual physical system will behave differently.

The advantage of being 'unit agnostic' (different units on input and output) is that a PID controller can be used to provide closed-loop control between any input signal and output signal, so long as their is a strong causal relationship.  Therefore, you may later use another PID controller to minimise the error of other system state variables.  It is likely that you will use the output of one PID controller as the demand signal for another PID controller (called, nested control).

Whilst it is good that a PID controller can be 'unit agnostic', there is a clear drawback.  Every PID controller requires careful implementation and calibration.  PID controllers have a reputation for being annoying to work with because there are plenty of parts to go wrong.  When they go wrong, they tend to oscillate or exhibit chaotic, hard to understand behaviour.  Therefore, it is important to only take small steps and make small changes.  It is important to approach implementing a PID controller in a methodical way - a way which builds up complexity and removes as many errors and oversights as possible.

### Step 1: Check your Error Signal

The first task is to determine and check an error signal (demand vs measurement):
 - Check the error signal looks sensible, for example:
     - what are you measuring, what is a sensible range of values?
     - does it go both positive and negative?
     - does positive and negative make sense (e.g. correlates to forward and backward?)
     - is the error signal stable?

### Step 2: Tuning your PID

A PID controller is going to take your **error signal** and transform it into a meaningful **output signal** to drive your system.  To create this transformation, a PID controller must be **calibrated**, or **tuned**.   You may find tuning a PID controller difficult.  You should find a lecture to help guide you in this process.  You may arrive at a controller which does not use all terms (P, I and D) - such as a PD controller, or just a P controller - it depends on your task requirements.

The transformation of error signal to output signal is created by setting the **P, I and D gains**.  It is best to work with one gain at a time.  Simply put:
- **P-gain** multiplies (or divides) the error signal to produce the output signal, and it is the first gain you should set.  It is the most straight forward mapping of error signal to output signal.  
    - You can think  of the P-Gain as an aggressive response (a shunt) to try to "jump" the output signal to correct for error.  When the error decreases, so will the effect of the P-Gain.  If the error is 0, your P-Gain will multiple out to 0 as well.  Therefore, P-Gain has less influence on your system in low-error conditions.
    - It is unlikely that you will find a perfect p-gain value.  Generally, it is better for your system to undershoot, rather than overshoot.  
        - Overshoot can quickly become oscillation.  Trying to fix this with D-Gain will only create worse oscillations.  
        - Undershoot results in a steady state error, which can be fixed with I-Gain.
- **D-gain** compensates the P-gain, by producing an effect based on the rate-of-change.  You may notice your P-gain causes overshoot, and the D-gain can help to counteract this.
    - You can think of the P-Gain as acting like a dampening spring which resists sudden movement.
    - Don't use D-Gain to try to fix persistent oscillation, this is not it's purpose.  It will only cause more oscillation.
    - Only use D-Gain to attempt to "smooth" the response of your system under significant changes in the demand.
- **I-gain** adds to the output signal when an error signal is sustained over time.  It effectively 'accumulates' error if the target has not been reached quickly enough.  The I-gain is normally the most sensitive, and therefore has a very small gain-value.  It is best to experiment with the I-gain last.  
    - You can think of the I-Gain as a gentle persistent force to correct for an error signal that has not been fixed by the P-Gain.  
    - However, when I-Gain goes wrong, it is the opposite of gentle!

To tune your PID controller, you need to actually work with the physical system to: 
- understand how it behaves (often, the behaviour, or relationship, is non-linear).
- understand the characteristics of measurement, demand, error signal and output.

Completing the tasks in the previous labsheets should have helped you to gain an understanding some of the complexity of the performance of your Romi.  For instance, you might already be aware that your left motor is slower than your right motor (or vice-versa) when given the same power.  Each Romi will be slightly different.  

### Important Questions to Consider

You should approach tuning your PID controller with patience and a step-by-step process.  Work on one motor at a time.  Use the Serial Plotter to inspect the various calculations.  Tuning your PID controllers well will really help the overall performance of your system.

When implementing a PID controller we must first ask:
- Is there a relationship between error signal (demand vs measurement) and output?

If the answer is yes, then your system may be able to regulate itself.   However, we must then ask or investigate the following question:
- What are the characteristics of demand, measurement, error signal, and output?
    - Think about this question carefully, sketch down some graphs you'd expect to see.
    - Make sure you understand what a given error signal should be transformed into (by a gain value) to produce a useful output signal.
        - For example, if you observe your error signal on the Serial Monitor (or plotter), work out what P-Gain value would produce a useful output (analogWrite needs a value between 0 : 255 ).
    - Make sure you report useful information back using `Serial.print()`, and make sure you can understand what you are observing.
    - Work with one gain at a time, and make sure the relationship is as you would expect.
    - You can get strange, half-working performance if your error signal is inverted.
